# Introduction

This notebook details a deep learning approach to image modeling using keras and tensorflow on the backend. Keras is a high-level API to build and train deep learning models. It's used for fast prototyping, advanced research, and productions. TensorFlow is a free and open-source software library for dataflow and differentiable programming across a range of tasks. It is a symbolic math library, and is also used for machine learning applications such as neural networks. Using keras and tensorflow together provides us with a simplified method of developing an image classification model. The general process to building this detector is as follows:

1. Obtain labeled data 
2. Identify the structure of our image data 
3. Prepare the data for modeling
4. Select the appropriate model parameters
5. Build the model 
6. Validate the model using metrics
7. Evaluate predictions 

Before we begin, we need to load the appropriate libraries needed to run the notenook.

In [2]:
# Load Libraries

from PIL import Image
import numpy as np
import pandas as pd
import os
from random import shuffle
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
import numpy as np
from keras.utils import to_categorical
from tqdm import tqdm
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Using TensorFlow backend.


# Obtain Data 

After loading the libraries, we then need to load the training and testing data. This data was sourced from a kaggle competition located here: https://www.kaggle.com/c/emotion-detection-from-facial-expressions/overview. The raw data can be found here: https://github.com/muxspace/facial_expressions. Due to the size limitations of github, the images will have to be stored in a separate location not contained in this repository. To run this notebook we will need to specify the directory of the images folder and the path to the csv file containing the labels. 

In [1]:
# Specify data locations
IMAGES = "/Users/chad/Projects/large-data-files/Emotion-Detecion/images" # Folder containing images
LABELS = '/Users/chad/Projects/large-data-files/Emotion-Detecion/data/legend.csv' # CSV with image file names and correct labels


# Identify the structure of our image data

The data is provided in two forms. First, there is a folder of the raw images to be processed. The second is a .csv file containing image labels for each file name in the images folder. In order to build this model, we need to load the images and append the appropriate labels to the data. Before we load the data, we want to know how we should format the height x width image data dimensions for inputting to a keras model.

In [4]:
# Want to know how we should format the height x width image data dimensions
# for inputting to a keras model
def get_size_statistics():
    heights = []
    widths = []
    img_count = 0
    for img in os.listdir(IMAGES):
        try:
            path = os.path.join(IMAGES, img)
            if "DS_Store" not in path:
                data = np.array(Image.open(path))
                heights.append(data.shape[0])
                widths.append(data.shape[1])
                img_count += 1
        except:
            continue
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))

get_size_statistics()

Average Height: 335.75410075089303
Max Height: 536
Min Height: 24


Average Width: 333.5470583946927
Max Width: 441
Min Width: 18


The results of the analysis show that our average image height is 335 and our average image width is 333. This means that when we fit our model we can select dimensions as high as these values and should not run into any major issues. I have already experimented with several data dimensions and determined that using lower values does not degrade model performance whereas using high values exponentially increased training time. As such, for the purpose of this analysis I will use a 30 x 30 data dimension when loading the images. 

# Prepare the data for modeling


The data for this analysis is contained within one folder. This means that before building the training data, we should set aside a subset of images to evaluate our model predictions after we build the model. The code below first loads the labeled csv file and cleans up the label (converts all strings to lower case for consistency). We then create a new column of integer codes based on this label. This is important as the keras to_categorical function requires an integer value to work. We then take a randomly selected 10% subset of this data frame for later testing. The test records are then removed from the training dataset and we reset the index of both data frames to account for this reduction. 

In [5]:
train = pd.read_csv(LABELS)
train.emotion = train.emotion.apply(lambda x: x.lower())
train.emotion = pd.Categorical(train.emotion)
train['code'] = train.emotion.cat.codes
test = train.sample(frac=0.1, replace=True, random_state=1) # take 10% of records to make predictions later
train = pd.concat([train, test]).drop_duplicates(keep=False).reset_index() # remove test from training set
test = test.reset_index()

Using the image function from keras preprocessing, we then load the images with the specified dimensions above (30 x 30 x 1) and convert them to an array. 

In [6]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(IMAGES+'/'+train['image'][i], target_size=(30,30,1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
    

100%|██████████| 12402/12402 [00:08<00:00, 1488.58it/s]


The last portion of the data preparation involves us converting the multi class labels into categorical columns that will be used during training and then creating a training set and validation set. 

In [7]:
X = np.array(train_image)
y = train['code'].values
y = to_categorical(y) # dummie code

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2) # takes 20% of records for y_test


# Select the appropriate model parameters

We will create a simple architecture with 2 convolutional layers, one dense hidden layer and an output layer.

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(30,30,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Train on 9921 samples, validate on 2481 samples
Epoch 1/5
9921/9921 [==============================] - 28s 3ms/step - loss: 0.9716 - acc: 0.6011 - val_loss: 0.6793 - val_acc: 0.7674
Epoch 2/5
9921/9921 [==============================] - 24s 2ms/step - loss: 0.7027 - acc: 0.7626 - val_loss: 0.5935 - val_acc: 0.7940
Epoch 3/5
9921/9921 [==============================] - 24s 2ms/step - loss: 0.6299 - acc: 0.7856 - val_loss: 0.5556 - val_acc: 0.8110
Epoch 4/5
9696/9921 [============================>.] - ETA: 0s - loss: 0.5892 - acc: 0.7997

Generate predictions on the randomly sampled 10% of images in the data folder. 

In [ ]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img(IMAGES+'/'+test['image'][i], target_size=(30,30,1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test_data = np.array(test_image)

In [ ]:
# making predictions
test['predicted'] = model.predict_classes(test_data)

In [ ]:
# Create predicted emotion column
ids = test[['emotion', 'code']].drop_duplicates()
ids.columns = ['predicted_emotion', 'code']
test = pd.merge(test, ids, how='left', left_on='predicted', right_on='code')
test = test[['user.id', 'image', 'emotion', 'code_x', 'predicted', 'predicted_emotion']]
test.columns =['user.id', 'image', 'emotion', 'emotion_code', 'predicted_code', 'predicted_emotion']

In [ ]:
y_pred = test['predicted_code']
y_true = test['emotion_code']
accuracy_score(y_true, y_pred)

In [ ]:
# Create a function to print image, true label, and prediction

def print_image(index):
    img = image.load_img(IMAGES+'/'+test['image'][index], target_size=(300,300,1), color_mode = "grayscale")
    plt.imshow(img, cmap = 'gist_gray')
    print('Model predicted ' + test['predicted_emotion'][index] + ' the true emotion is ' + test['emotion'][index])
    
    

In [ ]:
print_image(500)

In [ ]:
print_image(10)

In [ ]:
print_image(20)

In [ ]:
print_image(345)

In [ ]:
print_image(88)

In [ ]:
test.head(25)